# Tectonic Solver
_Solving a Tectonic Grid with Python._

The goal is to fill the grid with numbers, according to the following rules : 
* A 2 squares color block contains numbers 1 and 2. A 3 squares color block contains numbers 1, 2, 3. And so on...
* A number in a square cannot be in any neighbor square (including diagonally)

![Grid to solve](GrilleVide_s.jpeg)

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
import numpy as np #pour manipuler des matrices

### Defining the grid

In [2]:
#Parameters
width = 5
length = 11
nb_blocks = 12 #numbe of blocks in this particular grid
resGrid = np.zeros((length,width)) #resultGrid

Defining the blocks :

In [3]:
blocks = np.array([[1,2,2,3,4],[1,1,2,3,4],[1,5,5,3,3],[6,5,5,5,3],[6,6,7,8,8],[6,6,7,8,8],[9,9,7,7,8],[9,9,7,10,11],[12,9,10,10,10],[12,12,10,13,13],[12,12,13,13,13]])
#I actually want blocks numbers starting at 0 but don't want to type all over again
blocks = blocks - 1

In [4]:
blocks

array([[ 0,  1,  1,  2,  3],
       [ 0,  0,  1,  2,  3],
       [ 0,  4,  4,  2,  2],
       [ 5,  4,  4,  4,  2],
       [ 5,  5,  6,  7,  7],
       [ 5,  5,  6,  7,  7],
       [ 8,  8,  6,  6,  7],
       [ 8,  8,  6,  9, 10],
       [11,  8,  9,  9,  9],
       [11, 11,  9, 12, 12],
       [11, 11, 12, 12, 12]])

Setting given values :

In [5]:
fixed = np.array([[0,0,0,3,0],[0,0,0,0,0],[0,0,0,0,2],[0,0,2,0,0],[0,0,0,5,0],[0,2,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,5,0],[1,0,0,0,2],[0,5,0,4,0]])

In [6]:
fixed

array([[0, 0, 0, 3, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2],
       [0, 0, 2, 0, 0],
       [0, 0, 0, 5, 0],
       [0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 5, 0],
       [1, 0, 0, 0, 2],
       [0, 5, 0, 4, 0]])

*__Reminder --> all indexes start at Zero :__*

In [7]:
print(f'Number 2 in fourth row, third column is actually called with "fixed[3,2]" : {fixed[3,2]}')

Number 2 in fourth row, third column is actually called with "fixed[3,2]" : 2


### Class Cell
Each square is a Cell and has the following parameters and methods : 
* Parameters : own value, max value in its block, state, position in the grid, list of cells in its block, list of neighbors
* Methods : initialisation (sets position in the grid), tests if value is ok, method to fill value following the game rules

In [8]:
class Cell:

    maxi:int = None # Max value in block
    state:str = None # Status of the current value : fixed / notmax / maxed
    value:int = None # Value
    blocMembers:[Cell] = None #List of blockMembers
    neighbors:[Cell] = None #List of neighbors
    # Position in the grid
    posi:int = None 
    posj:int = None


    def __init__(self, i:int, j:int) -> None:
        #constructor
        self.posi = i
        self.posj = j
        self.neighbors = []
        
    def testBlockMembers (self) -> bool:
        #testing if value fits within the block
        for b in self.blocMembers:
            if self.value == b.value:
                return False
        return True
                
    def testNeighbors (self) -> bool:
        #testing if value fits within neighbors
        for n in self.neighbors:
            if self.value == n.value:
                return False
        return True
    
    def testValue (self) -> bool:
        #testng block and neighbors
        if self.testNeighbors() and self.testBlockMembers():
            return True
        else:
            return False
        
    def fill (self, sens:bool) -> int:
        # If this is a fixed value, nothing to do and keep going
        if self.state == "fixed":
            print(f'Cell : value not changed : {self.value}')
            if sens:
                print("Cell : fixed. Keep going!")
                return 1 #keepGoing
            else:
                print("Cell : fixed. Keep going backward!")
                return -1 #keepgoingBackward
        # If max is reached, set to zero and go backwards
        elif self.state == "maxed":
            print("Cell : maxed. Set to 0 and go back!")
            self.value = 0 
            self.state = "notmax"
            print(f'Cell : new value {self.value}, new status {self.state}')
            return -1 #goBack
        # Else add one and test
        elif self.state =="notmax":
            print("Cell : adding one")
            self.value +=1
            if self.value == self.maxi:
                    self.state="maxed"
                    print(f'Cell : max reached, new value : {self.value}, new state : {self.state}')
            else:
                    self.state="notmax"
                    print(f'Cell : max not reached, new value : {self.value}, new state : {self.state}')        
            if self.testValue() :
                print("Cell : test ok, we keep going.")
                return 1 #keepGoing
            else:
                print("Cell : test failed, we'll stay here")
                return 0 #stayHere

### Creating cells and putting them in a list

We instanciate as many cells as needed in the grid (length × width) and put them in a list.  
When instanciated each cell is given as parameters its position in the grid, so that we can later know who their neighbors are and what block they belong to.

In [9]:
cells=[] #list of cells
for i in range(length):
    for j in range(width):
        cells.append(Cell(i,j))

### Adding fixed values and statuses

Some values are given (we stored them in the "fixed" table).  
We set these values in the right cells and set their parameter accordingly to "fixed". 

In [10]:
for c in cells:
    c.value=fixed[c.posi,c.posj]
    if c.value == 0:
        c.state = "notmax"
    else:
        c.state= "fixed"

### Setting each cell's block list

The grid is made of blocks from 1 to 5 cells.  
We create a list of all existing blocks, and for each block a list of the cells the block is made of.

In [11]:
# Declaring list
blocks_lists=[[]]
for i in range(nb_blocks):
    blocks_lists.append([])
#blocks_lists

# Filling this list : for each cell, we add it to the list of its block
for i in range(length):
    for j in range(width):
        a = blocks[i,j]
        blocks_lists[a].append(cells[5*i+j])

We now give each cell its block list and its block size

In [12]:
# WARNING : we want to duplicate the list, otherwise all cells sharing a list will have the same reference
for i in range(length):
    for j in range(width):
        cells[5*i+j].blocMembers=blocks_lists[blocks[i,j]].copy()
# The max value for the cell is the lists length
for c in cells:
    c.maxi = len(c.blocMembers) #size of the bloc

We remove each cell from its own block (to facilitate tests)

In [13]:
# We remove each cell from its own blocMembers Cell list
for c in cells:
    i=0
    #print(f'cell C [{c.posi},{c.posj}]')
    for b in c.blocMembers:
        #print(f'cell B [{b.posi},{b.posj}]')
        if c.posi == b.posi and c.posj == b.posj:
            #print('Removing Cell!!')
            c.blocMembers.pop(i)
            break
        i+=1

### Setting each cell's neighbors list

Based on it's position, we're able to know which are a cell's neighbors.

In [14]:
for c in cells:
    # Adding neighbors
    for i in range(c.posi-1,c.posi+2,1):
        for j in range (c.posj-1,c.posj+2,1):
            if (i>=0 and i<length) and (j>=0 and j<width): 
                c.neighbors.append(cells[5*i+j])

    # Removing cell from its own list
    i=0
    for b in c.neighbors:
        if c.posi == b.posi and c.posj == b.posj:
            c.neighbors.pop(i)
            break
        i+=1

## Solving the Tectonic grid

In [15]:
i=0
k=0
direction = True
activeCell = cells[i]
lastCell = cells[length*width-1]

#Warning : if lastCell is a given number, the solver won't start!!
while (lastCell.value == 0 or lastCell.testValue()==False):
    k+=1
    print(f'STEP {k}')
    print(f'ActiveCell : {activeCell.posi},{activeCell.posj}, active i : {i}, value : {activeCell.value}')
    oldi = i
    i = i + activeCell.fill(direction)
    # We need to know if progressing forward or backward in the list
    if i<oldi:
        direction=False
    else:
        direction=True
    if i<len(cells):
        activeCell=cells[i]
    else:
        break

STEP 1
ActiveCell : 0,0, active i : 0, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test ok, we keep going.
STEP 2
ActiveCell : 0,1, active i : 1, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 3
ActiveCell : 0,1, active i : 1, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test ok, we keep going.
STEP 4
ActiveCell : 0,2, active i : 2, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test ok, we keep going.
STEP 5
ActiveCell : 0,3, active i : 3, value : 3
Cell : value not changed : 3
Cell : fixed. Keep going!
STEP 6
ActiveCell : 0,4, active i : 4, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test ok, we keep going.
STEP 7
ActiveCell : 1,0, active i : 5, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new s

Cell : test failed, we'll stay here
STEP 109
ActiveCell : 1,3, active i : 8, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test failed, we'll stay here
STEP 110
ActiveCell : 1,3, active i : 8, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test ok, we keep going.
STEP 111
ActiveCell : 1,4, active i : 9, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test ok, we keep going.
STEP 112
ActiveCell : 2,0, active i : 10, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 113
ActiveCell : 2,0, active i : 10, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test failed, we'll stay here
STEP 114
ActiveCell : 2,0, active i : 10, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test ok, we keep going

Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test ok, we keep going.
STEP 515
ActiveCell : 7,1, active i : 36, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 516
ActiveCell : 7,1, active i : 36, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test failed, we'll stay here
STEP 517
ActiveCell : 7,1, active i : 36, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test failed, we'll stay here
STEP 518
ActiveCell : 7,1, active i : 36, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test ok, we keep going.
STEP 519
ActiveCell : 7,2, active i : 37, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 520
ActiveCell : 7,2, active i : 37, value : 1
Cell : adding one
Cell : m

ActiveCell : 7,3, active i : 38, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test failed, we'll stay here
STEP 617
ActiveCell : 7,3, active i : 38, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test ok, we keep going.
STEP 618
ActiveCell : 7,4, active i : 39, value : 0
Cell : adding one
Cell : max reached, new value : 1, new state : maxed
Cell : test failed, we'll stay here
STEP 619
ActiveCell : 7,4, active i : 39, value : 1
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 620
ActiveCell : 7,3, active i : 38, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test failed, we'll stay here
STEP 621
ActiveCell : 7,3, active i : 38, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 622
ActiveCell : 7,2, active i : 37, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new sta

ActiveCell : 7,1, active i : 36, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 1055
ActiveCell : 7,0, active i : 35, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 1056
ActiveCell : 6,4, active i : 34, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test failed, we'll stay here
STEP 1057
ActiveCell : 6,4, active i : 34, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 1058
ActiveCell : 6,3, active i : 33, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 1059
ActiveCell : 6,2, active i : 32, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test failed, we'll stay here
STEP 1060
ActiveCell : 6,2, active i : 32, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test ok, we keep going.
STEP 1061
ActiveCell :

...

ActiveCell : 7,2, active i : 37, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test failed, we'll stay here
STEP 2197
ActiveCell : 7,2, active i : 37, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test failed, we'll stay here
STEP 2198
ActiveCell : 7,2, active i : 37, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test failed, we'll stay here
STEP 2199
ActiveCell : 7,2, active i : 37, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 2200
ActiveCell : 7,1, active i : 36, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test failed, we'll stay here
STEP 2201
ActiveCell : 7,1, active i : 36, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 2202
ActiveCell : 7,0, active i : 35, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 

Cell : test failed, we'll stay here
STEP 2541
ActiveCell : 8,1, active i : 41, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test failed, we'll stay here
STEP 2542
ActiveCell : 8,1, active i : 41, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test ok, we keep going.
STEP 2543
ActiveCell : 8,2, active i : 42, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 2544
ActiveCell : 8,2, active i : 42, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test ok, we keep going.
STEP 2545
ActiveCell : 8,3, active i : 43, value : 5
Cell : value not changed : 5
Cell : fixed. Keep going!
STEP 2546
ActiveCell : 8,4, active i : 44, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 2547
ActiveCell : 8,4, active i : 44, val

STEP 2704
ActiveCell : 8,0, active i : 40, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 2705
ActiveCell : 8,0, active i : 40, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test ok, we keep going.
STEP 2706
ActiveCell : 8,1, active i : 41, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test failed, we'll stay here
STEP 2707
ActiveCell : 8,1, active i : 41, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test failed, we'll stay here
STEP 2708
ActiveCell : 8,1, active i : 41, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test failed, we'll stay here
STEP 2709
ActiveCell : 8,1, active i : 41, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test failed, we'll stay here
STEP 2710
Activ

STEP 3028
ActiveCell : 7,3, active i : 38, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test failed, we'll stay here
STEP 3029
ActiveCell : 7,3, active i : 38, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 3030
ActiveCell : 7,2, active i : 37, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test failed, we'll stay here
STEP 3031
ActiveCell : 7,2, active i : 37, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test failed, we'll stay here
STEP 3032
ActiveCell : 7,2, active i : 37, value : 3
Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test failed, we'll stay here
STEP 3033
ActiveCell : 7,2, active i : 37, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test failed, we'll stay here
STEP 3034
ActiveCell : 7,2, active i : 37, value : 5
Cell 

Cell : adding one
Cell : max not reached, new value : 4, new state : notmax
Cell : test failed, we'll stay here
STEP 3175
ActiveCell : 6,1, active i : 31, value : 4
Cell : adding one
Cell : max reached, new value : 5, new state : maxed
Cell : test failed, we'll stay here
STEP 3176
ActiveCell : 6,1, active i : 31, value : 5
Cell : maxed. Set to 0 and go back!
Cell : new value 0, new status notmax
STEP 3177
ActiveCell : 6,0, active i : 30, value : 1
Cell : adding one
Cell : max not reached, new value : 2, new state : notmax
Cell : test failed, we'll stay here
STEP 3178
ActiveCell : 6,0, active i : 30, value : 2
Cell : adding one
Cell : max not reached, new value : 3, new state : notmax
Cell : test ok, we keep going.
STEP 3179
ActiveCell : 6,1, active i : 31, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new state : notmax
Cell : test ok, we keep going.
STEP 3180
ActiveCell : 6,2, active i : 32, value : 0
Cell : adding one
Cell : max not reached, new value : 1, new st

# Grid Solved :

In [16]:
for i in range(length*width):
    resGrid[i//5,i%5]=cells[i].value
resGrid

array([[1., 3., 1., 3., 2.],
       [2., 4., 2., 4., 1.],
       [3., 1., 3., 5., 2.],
       [4., 5., 2., 4., 1.],
       [3., 1., 3., 5., 3.],
       [5., 2., 4., 2., 1.],
       [1., 3., 1., 5., 4.],
       [2., 4., 2., 3., 1.],
       [3., 5., 1., 5., 4.],
       [1., 4., 2., 3., 2.],
       [2., 5., 1., 4., 5.]])